In [112]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pylab as pylab
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [113]:
%matplotlib inline
mpl.style.use('ggplot')
pylab.rcParams['figure.figsize'] = 9,5

In [114]:
df = pd.read_csv('data/heart.csv', usecols=['sex', 'cp', 'restecg000', 'exang', 'slope', 'thal', 'target000'])

In [115]:
df.head()

,sex,cp,restecg000,exang,slope,thal,target000
0,1,3,0,0,0,1,1
1,1,2,1,0,0,2,1
2,0,1,0,0,2,2,1
3,1,1,1,0,2,2,1
4,0,0,1,1,2,2,1


In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 7 columns):
sex           303 non-null int64
cp            303 non-null int64
restecg000    303 non-null int64
exang         303 non-null int64
slope         303 non-null int64
thal          303 non-null int64
target000     303 non-null int64
dtypes: int64(7)
memory usage: 16.7 KB


In [117]:
df.describe()

,sex,cp,restecg000,exang,slope,thal,target000
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,0.683168,0.966997,0.528053,0.326733,1.399340,2.313531,0.544554
std,0.466011,1.032052,0.525860,0.469794,0.616226,0.612277,0.498835
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000
50%,1.000000,1.000000,1.000000,0.000000,1.000000,2.000000,1.000000
75%,1.000000,2.000000,1.000000,1.000000,2.000000,3.000000,1.000000
max,1.000000,3.000000,2.000000,1.000000,2.000000,3.000000,1.000000


In [118]:
#df.hist(bins=50, figsize=(15, 10))

In [119]:
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)
print("Uczące", len(train_set), ", testowe", len(test_set))

Uczące 242 , testowe 61


In [120]:
train_labels = train_set.target000
train_set.drop("target000", axis=1, inplace=True)
test_labels = test_set.target000
test_set.drop("target000", axis=1, inplace=True)

In [121]:
X = train_set.values
d = train_labels.values

# Progowa unipolarna

In [122]:
class Perceptron:
    def __init__(self, input_size, lr=1, epochs=100):
        self.W = np.zeros(input_size+1)
        self.epochs = epochs
        self.lr = lr
    
    def activation_fn(self, x):
        if x > 0:
            return 1
        else:
            return 0
    
    def predict(self, x):
        z = self.W.T.dot(x)
        a = self.activation_fn(z)
        return a
 
    def fit(self, X, d):
        for _ in range(self.epochs):
            for i in range(d.shape[0]):
                x = np.insert(X[i], 0, 1)
                y = self.predict(x)
                e = d[i] - y
                self.W = self.W + self.lr * e * x

In [123]:
perceptron = Perceptron(input_size=6)
perceptron.fit(X, d)
print(perceptron.W)

[ 2. -5.  6.  2. -6.  4. -4.]


# Progowa bipolarna

In [124]:
class Perceptron:
    def __init__(self, input_size, lr=1, epochs=100):
        self.W = np.zeros(input_size+1)
        self.epochs = epochs
        self.lr = lr
    
    def activation_fn(self, x):
        if x > 0:
            return 1
        else:
            return - 1
    
    def predict(self, x):
        z = self.W.T.dot(x)
        a = self.activation_fn(z)
        return a
 
    def fit(self, X, d):
        for _ in range(self.epochs):
            for i in range(d.shape[0]):
                x = np.insert(X[i], 0, 1)
                y = self.predict(x)
                e = d[i] - y
                self.W = self.W + self.lr * e * x

In [125]:
perceptron = Perceptron(input_size=6)
perceptron.fit(X, d)
print(perceptron.W)

[ 4. -2.  8.  4. -4. -2. -2.]


# Sigmoidalna unipolarna

In [103]:
import math

In [126]:
class Perceptron:
    def __init__(self, input_size, lr=1, epochs=100, parameter=1):
        self.W = np.zeros(input_size+1)
        self.epochs = epochs
        self.lr = lr
        self.parameter = parameter
    
    def activation_fn(self, x):
        return 1 / (1 + math.e ** (self.parameter * - x))
    
    def predict(self, x):
        z = self.W.T.dot(x)
        a = self.activation_fn(z)
        return a
 
    def fit(self, X, d):
        for _ in range(self.epochs):
            for i in range(d.shape[0]):
                x = np.insert(X[i], 0, 1)
                y = self.predict(x)
                e = d[i] - y
                self.W = self.W + self.lr * e * x

In [129]:
perceptron = Perceptron(input_size=6, parameter=1)
perceptron.fit(X, d)
print(perceptron.W)

[ 2.95876863 -5.31666486  5.45320796  3.58511302 -5.25000145  4.8812447
 -4.65850318]


# Tangens hiperboliczny

In [130]:
class Perceptron1:
    def __init__(self, input_size, lr=1, epochs=100, parameter=1):
        self.W = np.zeros(input_size+1)
        self.epochs = epochs
        self.lr = lr
        self.parameter = parameter
    
    def activation_fn(self, x):
        return math.tanh((self.parameter * x) / math.e)
    
    def predict(self, x):
        z = self.W.T.dot(x)
        a = self.activation_fn(z)
        return a
 
    def fit(self, X, d):
        for _ in range(self.epochs):
            for i in range(d.shape[0]):
                x = np.insert(X[i], 0, 1)
                y = self.predict(x)
                e = d[i] - y
                self.W = self.W + self.lr * e * x

In [131]:
perceptron = Perceptron1(input_size=6, parameter=1)
perceptron.fit(X, d)
perceptron.W

array([ 6.56978088, -0.39856821,  3.80660474,  3.46005582, -2.89697209,
        0.69601818, -1.1564936 ])

In [132]:
X = train_set.values
Y = train_labels.values
test_data = test_set.values
test_labels = test_labels.values

In [133]:
from sklearn.metrics import accuracy_score

In [134]:
p_acc = 0

for i in range(100):

    p = Perceptron1(input_size=6, parameter=1)
    p = p.fit(X,Y)
    p_prediction = p.predict(test_data)
    #print(dtc_prediction)
    
p_acc += accuracy_score(p_prediction, test_labels)

AttributeError: 'NoneType' object has no attribute 'predict'